<img src="./images/DLI_Header.png" style="width: 400px;">

# 使用 MNIST 資料集進行影像分類

在本節中，我們會執行深度學習的「Hello World」：訓練深度學習模型以正確分類手寫數字。

## 目標

* 瞭解深度學習如何解決傳統程式設計方法無法解決的問題
* 瞭解 [MNSIT 手寫數字資料集](http://yann.lecun.com/exdb/mnist/)
* 使用 [Keras API](https://keras.io/) 載入 MNIST 資料集並準備進行訓練
* 建立簡單的神經網路以執行影像分類
* 使用準備充分的 MNIST 資料集訓練神經網路
* 觀察經過訓練的神經網路效能

## 問題：影像分類

在傳統的程式設計中，程式設計師能夠在程式碼裡清楚陳述規則和條件，讓程式能夠透過程式碼正確執行。這種方法長期以來都能有效解決各式各樣的問題。

影像分類指的是要求程式將從未看過的影像正確分類到對應的類別，但以傳統的程式設計技術幾乎不可能解決這項問題。程式設計師要如何定義規則和條件，才能正確分類各種影像，何況還要考量到他們從未看過的影像？

## 解決方案：深度學習

深度學習擅於透過試誤法來辨識模式。運用充分的資料來訓練深度神經網路，再為網路提供其訓練後效能的回饋，網路就可以經由大量迭代來自行辨識出一組條件，並依這些條件正確執行動作。

## MNIST 資料集

由 70,000 張從 0 到 9 手寫數字灰階影像組成的 [MNSIT 資料集](http://yann.lecun.com/exdb/mnist/)能夠實現準確的影像分類，是深度學習歷史上的一大進展。雖然現在這項問題並沒有被視為重大問題，但使用 MNIST 進行影像分類已成為深度學習的「Hello World」。

以下是 MNIST 資料集包含的其中 40 張影像：

<img src="./images/mnist1.png" style="width: 600px;">

## 訓練及驗證資料與標籤

使用影像進行深度學習時，我們不僅需要影像本身，通常以 `X`表示，還需要這些影像的正確[標籤](https://developers.google.com/machine-learning/glossary#label)，通常以 `Y` 表示。此外，我們也需要 `X`和 `Y` 兩者的值來*訓練*模型，以及另一組 `X` 和 `Y` 的值來*驗證*模型經過訓練後的效能。因此，我們需要 4 種 MNIST 資料集的資料段：

1.`x_train`：用於訓練神經網路的影像
2.`y_train`：`x_train` 影像的正確標籤，用於評估模型在訓練過程中的預測表現
3.`x_valid`：另外一組影像，用於驗證模型經過訓練後的效能
4.`y_valid`：`x_valid` 影像的正確標籤，用於評估模型經過訓練後的預測表現

準備分析資料的過程稱為[資料工程](https://medium.com/@rchang/a-beginners-guide-to-data-engineering-part-i-4227c5c457d7)。若要深入瞭解訓練資料與驗證資料之間的差異 (以及測試資料)，請參閱 Jason Brownlee 的[這篇文章](https://machinelearningmastery.com/difference-test-validation-datasets/)。

## 將資料載入記憶體 (透過 Keras)

[深度學習框架](https://developer.nvidia.com/deep-learning-frameworks)有多種類型，且各有不同的優點。在此實作坊中，我們會使用 [Tensorflow 2](https://www.tensorflow.org/tutorials/quickstart/beginner)，尤其會用到 [Keras API](https://keras.io/)。Keras 包含許多專為電腦視覺工作所設計的內建功能。這也是在專業環境下進行深度學習的合理選擇，因為 Keras [淺顯易懂](https://blog.pragmaticengineer.com/readable-code/)又高效率，不過其他深度學習框架同樣也有這些優點，因此相當建議在展開深度學習專案時深入瞭解各種框架。

Keras 提供的眾多實用功能之一是包含許多 Helper 方法的模組，適用於[許多常用的資料集](https://www.tensorflow.org/api_docs/python/tf/keras/datasets)，包括 MNIST。

首先，我們會載入 MNIST 的 Keras 資料集模組：

In [ ]:
from tensorflow.keras.datasets import mnist

有了 `mnist`模組，我們就能輕鬆載入 MNIST 資料，且資料已分割成影像和標籤，可用於訓練與驗證：

In [ ]:
# the data, split between train and validation sets
(x_train, y_train), (x_valid, y_valid) = mnist.load_data()

## 探索 MNIST 資料

我們先前有提到，MNIST 資料集包含 70,000 張手寫數字灰階影像。執行下列儲存格之後，我們可以看到 Keras 已經分割其中的 60,000 張影像用於訓練，另外的 10,000 張則用於驗證 (訓練之後)，而每個影像本身是尺寸 28x28 的 2D 陣列：

In [ ]:
x_train.shape

In [ ]:
x_valid.shape

此外，我們可以看到這些 28x28 影像的呈現方式為一系列介於 0 到 255 的 8 bits unsinged integer (正整數值)，這些值對應到像素的灰階值，其中 `0`代表黑色，`255` 代表白色，其他所有值都介於兩者之間：

In [ ]:
x_train.dtype

In [ ]:
x_train.min()

In [ ]:
x_train.max()

In [ ]:
x_train[0]

我們可以使用 [Matplotlib](https://matplotlib.org/) 在資料集中，視覺化出其中一張灰階影像：

In [ ]:
import matplotlib.pyplot as plt

image = x_train[0]
plt.imshow(image, cmap='gray')

這樣一來，我們就可以看到這是一張值為 5 的 28x28 像素影像。還是 3？答案就在 `y_train`資料中，其中包含資料的正確標籤。我們來看看：

In [ ]:
y_train[0]

## 準備資料以進行訓練

在深度學習中，經常需要將資料轉換為適合訓練的狀態。針對這個特定的影像分類問題，在準備訓練時，我們應該對資料執行 3 項工作：
1.扁平化 (Flatten) 影像資料，以簡化輸入至模型的影像資料
2.正規化影像資料，讓影像輸入值更適合用於模型
3.分類標籤，讓標籤值更適合用於模型

### 扁平化 (Flatten) 影像資料

雖然深度學習模型可以接受 2 維影像 (在我們的例子中為 28x28 像素)，我們還是會簡化最初用到的資料，將每張影像[重新調整](https://www.tensorflow.org/api_docs/python/tf/reshape)為 784 連續像素的單一陣列 (註：28x28 = 784)。這也就是所謂的扁平化 (Flatten) 影像。

在這裡我們是使用 Helper 方法 `reshape`來完成這項作業：

In [ ]:
x_train = x_train.reshape(60000, 784)
x_valid = x_valid.reshape(10000, 784)

我們可以確認影像資料已經過重新調整，現在是一系列各自包含 784 像素值的 1D 陣列：

In [ ]:
x_train.shape

In [ ]:
x_train[0]

### 正規化影像資料

深度學習模型比較擅於處理介於 0 和 1 的浮點數字 (稍後會針對這個主題詳細說明)。將整數值轉換為介於 0 和 1 的浮點值就叫做[正規化](https://developers.google.com/machine-learning/glossary#normalization)，我們在這裡會用簡單的方法來常規化資料，也就是將所有像素值 (如果你還記得的話，值都是介於 0 到 255) 除以 255：

In [ ]:
x_train = x_train / 255
x_valid = x_valid / 255 

現在我們可以看到這些值都是介於 `0.0`和 `1.0` 的浮點值：

In [ ]:
x_train.dtype

In [ ]:
x_train.min()

In [ ]:
x_train.max()

### 分類編碼

想想看，如果我們的問題是：7-2 等於多少？ 4 會比 9 更接近正確答案。然而，針對這種影像分類問題，我們不希望神經網路學會的是這種推理邏輯：我們只想讓神經網路選出正確的類別，並理解如果其中有數字 5 的影像，那麼猜出 4 和猜出 9 都是一樣錯誤的答案。

目前來說，影像的標籤是 0 到 9 之間的整數。由於這些值代表數字範圍，因此模型可能會試圖根據其猜測的數字和正確數字類別的接近程度，來推導出與其效能相關的結論。

因此，我們必須用分類編碼的方式來處理我們的資料。這種類型的轉換會修改資料，讓每個值都可以代表所有可能類別，而這個特定值的真正類別則會被設定為 true。

現在來看看簡單的範例，假設我們有 3 個類別：紅色、藍色和綠色。以任一顏色來說，其中 2 個類別會是 false，剩下的一個類別則為 true：

|實際色彩| 是紅色嗎？| 是藍色嗎？| 是綠色嗎？|
|------------|---------|----------|----------|
|紅色|True|False|False|
|綠色|False|False|True|
|藍色|False|True|False|
|綠色|False|False|True|

我們可以不使用「True」或「False」，而改用二進位表示相同的意義，也就是 1 或 0：

|實際色彩| 是紅色嗎？| 是藍色嗎？| 是綠色嗎？|
|------------|---------|----------|----------|
|紅色|1|0|0|
|綠色|0|0|1|
|藍色|0|1|0|
|綠色|0|0|1|

這就是分類編碼，將那些應該要被視為分類標籤的值轉換成另一種表示形式，讓模型能夠清楚理解分類的本質。因此，如果我們要使用這些值來進行訓練，就會將...

```python
values = ['red, green, blue, green']
```

...神經網路難以理解的資料，轉換為：

```python
values = [
    [1, 0, 0],
    [0, 0, 1],
    [0, 1, 0],
    [0, 0, 1]
]
```

### 對標籤進行分類編碼

Keras 提供的公用程式可以[對值進行分類編碼](https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical)，在這裡我們會使用這個程式來針對訓練和驗證標籤執行分類編碼：

In [ ]:
import tensorflow.keras as keras
num_categories = 10

y_train = keras.utils.to_categorical(y_train, num_categories)
y_valid = keras.utils.to_categorical(y_valid, num_categories)

以下是訓練標籤的前 10 個值，你可以看到都已經過分類編碼：

In [ ]:
y_train[0:9]

## 建立模型

準備好訓練用的資料之後，現在要開始建立模型，並使用資料進行訓練。第一個基本模型會由多*層*組成並有 3 個主要部分：

1.輸入層，負責接收某種預期格式的資料
2.數個[隱藏層](https://developers.google.com/machine-learning/glossary#hidden-layer)，各由多個*神經元*組成。每個[神經元](https://developers.google.com/machine-learning/glossary#neuron)都會依*權重*影響到網路的猜測，權重值會隨著網路獲得效能和學習的回饋而在多次迭代中更新。
3.輸出層，負責呈現網路對特定影像的猜測

### 具現化模型

首先，我們要使用 Keras 的[循序](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) 模型類別來具現化模型的執行個體，模型包含一連串的"層"讓資料得以按順序傳遞：

In [ ]:
from tensorflow.keras.models import Sequential

model = Sequential()

### 建立輸入層

接下來，我們要新增輸入層。這一層會是*Dense*，表示其中的每個神經元及其權重，都會影響下一層的每個神經元。為了透過 Keras 做到這一點，我們會使用 Keras 的[Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)層類別。

In [ ]:
from tensorflow.keras.layers import Dense

`units`引數會指定這一層的神經元數量。我們要使用 `512`，這是我們從實驗中選出的數量。必須選擇正確的神經元數量，就是「資料科學」為什麼是一門「科學」的原因，因為這需要掌握資料集的統計複雜度。稍後再試驗一下這個值，看看對訓練有什麼影響，並開始理解這個數字所代表的意義。

稍後我們會進一步瞭解激活函數，但現在我們要先使用 `relu`激活函數，簡而言之，相較於根據某些嚴格的線性函數來進行推測，這個函數有助於我們的網路學會如何對資料進行更複雜的推測。

`input_shape`值會指定傳入資料的形狀，以我們的情況而言，就是有 784 個值的 1D 陣列：

In [ ]:
model.add(Dense(units=512, activation='relu', input_shape=(784,)))

### 建立隱藏層

現在，我們要新增另一個Dense層。同樣地，我們稍後會更詳細說明，但現在只需要知道這些層為網路提供更多參數以進行推測，因此有更大的機會學習準確：

In [ ]:
model.add(Dense(units = 512, activation='relu'))

### 建立輸出層

最後，我們要新增輸入層。這一層會使用激活函數 `softmax`，這會使層中每一個值成為介於 0 和 1 的機率，因此這一層所有輸出的總和會是 1。在這種情況下，由於網路是要猜測單一影像屬於 10 種可能類別中的哪 1 種，因此共會有 10 個輸出結果。模型推測影像屬於特定類別之後，每個輸出都會為模型的推測提供一個機率：

In [ ]:
model.add(Dense(units = 10, activation='softmax'))

### 總結模型

Keras 會提供模型的呼叫函式[Summary](https://www.tensorflow.org/api_docs/python/tf/summary)， 可以印出模型的摘要：

In [ ]:
model.summary()

請注意可訓練參數的數量。每種參數都可以在訓練期間調整，對於訓練好的模型推測也有影響。

### 編譯模型

同樣地，之後會補充更多細節，但我們在實際用資料訓練模型之前，要採取的最後一個步驟就是[編譯](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential#compile)模型。我們會在這裡指定[損失函數](https://developers.google.com/machine-learning/glossary#loss)，用途是讓模型理解其在訓練過程中的表現效果如何。我們也會指定要在模型進行訓練時追蹤 `accuracy`：

In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

## 訓練模型

現在我們已經準備好訓練和驗證資料以及模型，可以開始使用訓練資料來訓練模型，再用驗證資料來驗證模型。

「使用資料訓練模型」通常也稱為「將模型與資料擬合」。換句話說，這突顯出模型的形狀會隨時間變化，以更準確地瞭解其所獲得的資料。

透過 Keras 擬合 (訓練) 模型時，我們會使用模型的[擬合](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit)方法。需要用到的引數如下：

* 訓練資料
* 訓練資料的標籤
* 應在整個訓練資料集上訓練的次數 (稱為 *Epoch*)
* 驗證或測試資料及其標籤

執行下方儲存格來訓練模型。訓練完成後我們會討論其輸出結果：

In [ ]:
history = model.fit(
    x_train, y_train, epochs=5, verbose=1, validation_data=(x_valid, y_valid)
)

### 觀察準確度

針對 5 個 Epoch 中的每個 Epoch，都需要注意 `accuracy`和 `val_accuracy` 分數。從 `accuracy`可以看出模型在這個 Epoch 對所有訓練資料的表現如何。從 `val_accuracy`可以看出模型在驗證資料上的表現，如果你還記得的話，驗證資料完全沒有用於訓練模型。

模型的表現不錯！準確度很快就接近 100%，驗證準確度也是一樣。現在我們的模型已經可以用來準確偵測和分類手寫影像。

下一步是使用這個模型來分類尚未看過的全新手寫影像。這個步驟稱為[推論](https://blogs.nvidia.com/blog/2016/08/22/difference-deep-learning-training-inference-ai/)。我們會在稍後的練習中探討推論的過程。

## 總結

我們的成果值得花點時間欣賞一下。過去，為進行這類的工作而打造的專業系統極為複雜，相關人員須投入整個職業生涯來打造系統 (請參閱[官方 MNIST 頁面](http://yann.lecun.com/exdb/mnist/)的參考資料，瞭解達成各里程碑費時了多少年)。

MNIST 的實用性不僅對電腦視覺有歷史性的影響，同時也是絕佳的[基準](http://www.cs.toronto.edu/~serailhydra/publications/tbd-iiswc18.pdf)和偵錯工具。每當新設計的機器學習架構難以正常運作？透過 MNIST 進行檢查吧。如果架構無法學習此資料集，則很可能無法學習更複雜的影像和資料集。

## 清除記憶體

在繼續之前，請執行下列儲存格以清除 GPU 記憶體。必須先完成此步驟才能繼續進行下一個 Notebook。

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## 下一步

在本節中，你學會了如何建立並訓練簡單的神經網路以進行影像分類。在下一節，你必須建立自己的神經網路，並執行資料準備以解決不同的影像分類問題。

請繼續前往 [02_asl.ipynb](02_asl.ipynb).

## ☆ 額外練習 ☆

還有點時間嗎？在下一節，我們會談到我們是如何得到上述的幾個數字，不過我們也可以試著想像一下，研究人員開發出當今常用技術的過程是如何。

用最簡單的方式來說，每個神經元都在試著把部分資料連成一條線。如下所示，我們有一些資料點和利用方程式 [y = mx + b](https://www.mathsisfun.com/equation_of_line.html) 隨機繪製的線條。

請嘗試變更 `m`與 `b`，盡可能找出最低的損失。要如何找到最理想的線？你可以寫出程式來落實你的策略嗎？

In [ ]:
import numpy as np
from numpy.polynomial.polynomial import polyfit
import matplotlib.pyplot as plt

m = -2  # -2 to start, change me please
b = 40  # 40 to start, change me please

# Sample data
x = np.array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9])
y = np.array([10, 20, 25, 30, 40, 45, 40, 50, 60, 55])
y_hat = x * m + b

plt.plot(x, y, '.')
plt.plot(x, y_hat, '-')
plt.show()

print("Loss:", np.sum((y - y_hat)**2)/len(x))

有什麼想法嗎？非常好！請先關閉核心，再繼續下一步。

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)